### QUESTION 1

In [3]:
pip install locationiq

Note: you may need to restart the kernel to use updated packages.


In [4]:
from locationiq.geocoder import LocationIQ
from math import radians, sin, cos, acos

In [5]:
geocoder = LocationIQ("37d68eb32b3b4d")

In [6]:
states = ['Agartala','Aizawl','Amaravathi','Bangalore','Bhopal','Bhubaneswar','Chandigarh','Chennai',
          'Dehradun','Dispur','Hyderabad','Imphal','Jaipur','Kohima','Kolkata','Lucknow','Mumbai','Patna',
          'Raipur','Shimla','Srinagar','Panaji','Thiruvanathapuram','Gandhinagar','Ranchi','Shillong','Itanagar',
          'Gangtok',]
print(states)

['Agartala', 'Aizawl', 'Amaravathi', 'Bangalore', 'Bhopal', 'Bhubaneswar', 'Chandigarh', 'Chennai', 'Dehradun', 'Dispur', 'Hyderabad', 'Imphal', 'Jaipur', 'Kohima', 'Kolkata', 'Lucknow', 'Mumbai', 'Patna', 'Raipur', 'Shimla', 'Srinagar', 'Panaji', 'Thiruvanathapuram', 'Gandhinagar', 'Ranchi', 'Shillong', 'Itanagar', 'Gangtok']


In [7]:
import json
distance = []
for i in range(len(states)):
    bg = geocoder.geocode('Bangalore')[0]
    ch = geocoder.geocode(states[i])[0]
    slat = radians(float(bg['lat']))
    slon = radians(float(bg['lon']))
    elat = radians(float(ch['lat']))
    elon = radians(float(ch['lon']))
    dist = 6370.01 * acos(sin(slat)*sin(elat) + cos(slat)*cos(elat)*cos(slon - elon))
    dist1 = int(dist)
    distance.append(dist1)
print(distance)

[1879, 1992, 307, 0, 1143, 1194, 1975, 291, 1929, 2084, 1682, 2160, 1560, 2240, 1560, 1579, 834, 1610, 1013, 2015, 2362, 492, 499, 1252, 1414, 2048, 2290, 1964]


In [8]:
heuristic_func= {} 
for key in states: 
    for value in distance: 
        heuristic_func[key.lower().strip()] = value 
        distance.remove(value) 
        break
print(heuristic_func)

{'agartala': 1879, 'aizawl': 1992, 'amaravathi': 307, 'bangalore': 0, 'bhopal': 1143, 'bhubaneswar': 1194, 'chandigarh': 1975, 'chennai': 291, 'dehradun': 1929, 'dispur': 2084, 'hyderabad': 1682, 'imphal': 2160, 'jaipur': 1560, 'kohima': 2240, 'kolkata': 1560, 'lucknow': 1579, 'mumbai': 834, 'patna': 1610, 'raipur': 1013, 'shimla': 2015, 'srinagar': 2362, 'panaji': 492, 'thiruvanathapuram': 499, 'gandhinagar': 1252, 'ranchi': 1414, 'shillong': 2048, 'itanagar': 2290, 'gangtok': 1964}


In [9]:
import xlrd
fileread = ("Indian_capitals.xlsx")
file = xlrd.open_workbook(fileread)
sheet = file.sheet_by_index(0)
main_function = []
for a in range(sheet.nrows):
    main_function.append(sheet.row_values(a))

In [10]:
destination = 'bangalore'

In [11]:
def testGoal(nodeState):
    if(nodeState == destination):
        return True
    else:
        return False

In [12]:
nodeChildren = dict()

for row in main_function:
    nodeChildren[(row[0].lower().strip(),row[1].lower().strip())] = [row[2]]

In [13]:
def findTheNeighbour(node):
    children = []
    for keys in nodeChildren:
        if node in keys:
            if keys[0] == node:
                child = keys[1]
                
            else:
                child = keys[0]
            children.append(child)
    return children

In [14]:
def createGraph(row):
    main = dict()
    for a in main_function:
        if a[0] not in main:
            main[a[0].lower().strip()] = findTheNeighbour(a[0].lower())
        if a[1] not in main:
            main[a[1].lower().strip()] = findTheNeighbour(a[1].lower())
    return main

graph = createGraph(main_function)
print(graph)

{'agartala': ['aizawl', 'dispur'], 'aizawl': ['agartala', 'imphal', 'dispur'], 'imphal': ['aizawl', 'dispur', 'kohima'], 'amaravathi': ['bangalore', 'chennai', 'bhubaneswar', 'raipur'], 'bangalore': ['amaravathi', 'panaji', 'chennai', 'thiruvanathapuram', 'mumbai', 'hyderabad'], 'chennai': ['amaravathi', 'bangalore', 'thiruvanathapuram'], 'bhubaneswar': ['amaravathi', 'raipur', 'ranchi', 'kolkata'], 'raipur': ['amaravathi', 'bhubaneswar', 'hyderabad', 'mumbai', 'bhopal', 'lucknow', 'ranchi'], 'panaji': ['bangalore', 'mumbai'], 'thiruvanathapuram': ['bangalore', 'chennai'], 'mumbai': ['bangalore', 'hyderabad', 'panaji', 'gandhinagar', 'bhopal', 'raipur'], 'bhopal': ['gandhinagar', 'jaipur', 'lucknow', 'mumbai', 'raipur'], 'gandhinagar': ['bhopal', 'jaipur', 'mumbai'], 'ranchi': ['bhubaneswar', 'kolkata', 'lucknow', 'patna', 'raipur'], 'kolkata': ['bhubaneswar', 'ranchi', 'patna', 'gangtok', 'dispur'], 'chandigarh': ['lucknow', 'jaipur', 'shimla', 'srinagar'], 'lucknow': ['chandigarh', '

In [15]:
def priority_queue(q, heuristic, priority_cost):
    priority_q = []
    for a in range(len(q)):
        min_ = q[a]        
        for b in range(len(q)):
            if q[b] in priority_q:
                continue
            if (priority_cost[min_[-1]] + heuristic[min_[-1]]) > (priority_cost[q[b][-1]] + heuristic[q[b][-1]]):
                min_ = q[b]
        priority_q.append(min_)
    return priority_q

In [16]:
def Searching(graph,source,testGoal,searchtype,heuristic, node_children):
    explored = []
    frontier = []
    start = [[source]]
    if(testGoal(source)):
        return 'starting state same as goal state'
    priority_cost = dict()
    priority_cost[source] = 0
    start = priority_queue(start,heuristic,priority_cost)
    path = []
    while start:
        path = start[0]
        start = start[1:]
        present_node = path[-1]
        if present_node not in explored:
            explored.append(present_node)
            children = graph[present_node.lower()]
            a = 0
            for child in children:
                if child not in frontier:
                    frontier.append(child)
                shortpath = list(path)
                if child not in shortpath:
                    shortpath.append(child)
                if(searchtype == 'bfs'):
                    start.append(shortpath)
                elif(searchtype == 'dfs'):
                    start.insert(a,shortpath)
                    a = a + 1
                else:
                    start.append(shortpath)
                    if ((present_node, child) in node_children):
                        temp = node_children[(present_node, child)][0]
                    else:
                        temp = node_children[(child, present_node)][0]                    
                    priority_cost[child] = priority_cost[present_node] + float(temp)
                    start = priority_queue(start, heuristic, priority_cost)
                if(testGoal(child)):
                    return shortpath
    return False

In [17]:
def BiDirectional(graph, start_point, testGoal,searchtype,heuristic_func, node_children):
    explored1 = []
    frontier1 = []
    explored2 = []
    frontier2 = []
    p1 = [[start_point]]
    p2 = [[destination]]
    if start_point == destination:
        return 'starting state same as goal state'
    else:
        while p1 != p2:
            path1 = p1[0]
            p1 = p1[1:]
            present_node1 = path1[-1]
            path2 = p2[0]
            p2 = p2[1:]
            present_node2 = path2[-1]
            if present_node1 not in explored1:
                explored1.append(present_node1)
                children = graph[present_node1.lower()]
                for child in children:
                    if child in frontier1:
                        continue
                    frontier1.append(child)
                    shortpath1 = list(path1)
                    if child in shortpath1:
                        continue
                    shortpath1.append(child)
                    p1.append(shortpath1)
            if present_node2 not in explored2:
                explored2.append(present_node2)
                children = graph[present_node2.lower()]
                for child in children:
                    if child in frontier2:
                        continue
                    frontier2.append(child)
                    shortpath2 = list(path2)
                    if child in shortpath2:
                        continue
                    shortpath2.append(child)
                    p2.append(shortpath2)
            if p1 == p2:
                path = shortpath2.reverse()
                for i in range(len(path)):
                    shortpath1.append(i)
                return shortpath
        return False 

In [18]:
source = input('Enter source City : ')
searchtype = input('Enter search type (bfs,dfs,astar,bidirectional) : ')
print(Searching(graph,source.lower(),testGoal,searchtype.lower(),heuristic_func, nodeChildren))

Enter source City : lucknow
Enter search type (bfs,dfs,astar,bidirectional) : bfs
['lucknow', 'bhopal', 'mumbai', 'bangalore']
